# Example of making animations from images of a Web Map Service

This examples requires 
* Julia 1.0 (https://julialang.org/downloads/)
* the julia package `Requests`, installable from the Julia command prompt:
```julia
Pkg.add("HTTP")
```
* the tool `ffmpeg` (https://www.ffmpeg.org/download.html)

In [6]:
using HTTP
using Base64

In [ ]:
# WMS base URL
wmsurl = "http://ec.oceanbrowser.net/emodnet-combined/Python/web/wms"

# WMS layer name
#layer = "Water_body_dissolved_oxygen_concentration.combined.nc*Water_body_dissolved_oxygen_concentration_L2"
layer = "Water_body_phosphate.combined.nc*Water_body_phosphate_L2"

# WMS bounding box (west,south,east,north)
bbox = "-18,30,42,66"

# short name of the title and file name of the animation
#shortname = "surface dissolved oxygen"
shortname = "surface phosphate"

# file name of the animation (without file extension)
animname = "emodnet-chemistry-$(replace(shortname," " => "-"))"

# frames per second
rate = 2

# colorbar range
vmin = 0
vmax = 0.8

# year range
years = 1980:2009;
#years = 2009:2010;

# all time instances for the animation
times = ["$season $year" for season in ["winter","spring","summer","autumn"],
         year in years][:];

# loop over all time frames
for i = 1:length(times)    
    param = Dict(
        "layers" => layer,
        "request" => "GetMap",
        "width" => "800",
        "height" => "500",
        "bbox" => bbox,
        "transparent" => "false",
        "decorated" => "true",
        "crs" => "CRS:84",
        "version" => "1.3.0",
        "styles" => "cmap:jet+inverted:false+method:pcolor_flat+vmin:$(vmin)+vmax:$(vmax)+ncontours:40",
        "format" => "image/png",
        "elevation" => "0.0",
        "time" => times[i],
        "title" => "$(shortname) ($(times[i]))",
        "basemap" => "shadedrelief")

    r = HTTP.get(wmsurl; query = param)

    # write into file
    open(@sprintf("%s-%05d.png",animname,i),"w") do f
        write(f, r.body)
    end
end

# generate the animation
run(`ffmpeg -y -r $(rate) -i $(animname)-%05d.png  -c:v libx264 -preset slow -crf 22 -pix_fmt yuv420p $(animname).mp4`)

Embed the animation in this notebook.

In [ ]:
display("text/html", string("""<video autoplay controls><source src="data:video/mp4;base64,""",
        base64encode(open(read,"$(animname).mp4")),"""" type="video/mp4"></video>"""));
